# Auxiliary Telescope Motor Condition Analysis


This notebook is used to extract the commanded positions, trajectories, velocities, and motor torques for each axis of the Main Telescope for a given time interval. By inputting a time interval, this notebook allows the rapid determination of what caused the fault, particularily in the case of motor slippage, or if a drive was commanded in an unstable fashion. <br>

It is expected that the user interacts with the Bokeh plots to better pinpoint the issue they're searching for. The plot ranges as created will most likely be too large, or contain too much data to be useful with zero manipulation of the axes/zooms etc. <br>

This notebook extracts data from the DM-EFD using [aioinflux](https://aioinflux.readthedocs.io/en/stable/index.html), a Python client for InfluxDB, and proceed with data analysis using Pandas dataframes. 

This is complementaty to the [Chronograf](https://test-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

In addition to `aioinflux`, you'll need to install `pandas`, `numpy` and `matplotlib` to run this notebook.

In [1]:
import matplotlib
%matplotlib widget
from matplotlib import pylab as plt
import aioinflux
import getpass
import pandas as pd
import asyncio
import numpy as np
from astropy.time import Time, TimeDelta

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
output_notebook()
from bokeh.models import Span, Label

from lsst_efd_client import EfdClient, resample

Loading BokehJS ...

In [2]:
client = EfdClient('summit_efd')
client.output = 'dataframe'

We'll access the DM-EFD instance deployed at the summit. You need to be on site or connected to the NOAO VPN. 

## Declare timestamps used for EFD queries

In [6]:
### Example #1 - 
t1 = Time('2023-03-08T03:40:01', scale='utc') 
window = TimeDelta(300, format='sec')
t2=t1+window

In [26]:
# Declare offset to tai
# Used in converting pointing vector data which is in TAI
utc_to_tai_offset = TimeDelta(37, format='sec')

In [7]:
# Find images taken during the time interval
image_info = await client.select_time_series("lsst.sal.GenericCamera.logevent_endReadout", 
                                             ["imageNumber", "timestampAcquisitionStart", "timestampEndOfReadout", "requestedExposureTime"], 
                                             t1, t2)
#print(image_info)

In [25]:
image_info

,imageNumber,timestampAcquisitionStart,timestampEndOfReadout,requestedExposureTime
2023-03-08 03:40:03.279500+00:00,357,1.678247e+09,1.678247e+09,4
2023-03-08 03:40:04.266997+00:00,357,1.678247e+09,1.678247e+09,5
2023-03-08 03:40:04.271420+00:00,357,1.678247e+09,1.678247e+09,6
2023-03-08 03:40:12.147449+00:00,358,1.678247e+09,1.678247e+09,4
2023-03-08 03:40:13.147225+00:00,358,1.678247e+09,1.678247e+09,5
2023-03-08 03:40:13.149307+00:00,358,1.678247e+09,1.678247e+09,6
2023-03-08 03:40:39.999638+00:00,359,1.678247e+09,1.678247e+09,4
2023-03-08 03:40:40.986044+00:00,359,1.678247e+09,1.678247e+09,5
2023-03-08 03:40:40.998830+00:00,359,1.678247e+09,1.678247e+09,6
2023-03-08 03:40:48.893306+00:00,360,1.678247e+09,1.678247e+09,4


In [10]:
# time check verification
print(t1.isot)
print(t1.datetime64)
print(t2.isot)

2023-03-08T03:40:01.000
2023-03-08T03:40:01.000000000
2023-03-08T03:45:01.000


# Azimuth Analysis

In [45]:
# query angle of azimuth (4 individual encoder heads)
az = await client.select_time_series("lsst.sal.MTMount.encoder",
                                      ["azimuthEncoderPosition0", "azimuthEncoderPosition1", "azimuthEncoderPosition2", "azimuthEncoderPosition3"], 
                                      t1, t2)

In [46]:
# query the angle commanded by the MT Pointing Component 
commanded_az_ATPng = await client.select_time_series("lsst.sal.MTMount.command_trackTarget", 
                                                     ["azimuth","private_sndStamp", "private_rcvStamp"], 
                                                      t1, t2)

In [16]:
# Useful code snippet on how to convert the time strings to a readable format
# help(pd.to_datetime(commanded_az_ATPng['private_sndStamp'][0], unit='s'))

In [35]:
def show_image_boundaries(image_info, yaxis_data):
    if len(image_info) > 0:
        # Generic camera selection #0, #1, or #2. 
        cam = 0
        for l in range(cam, len(image_info), 3):
            # NEED to check the time tai to utc in plot
            start = Time(image_info.timestampAcquisitionStart[l], format='unix', scale='tai') + utc_to_tai_offset
            #finish = Time(image_info.timestampEndOfReadout[l], format='unix', scale='tai') # issue with timestamps in astropy
            finish = start+TimeDelta(image_info.requestedExposureTime[l], format='sec')  # workaround
            # print(f"start for {l} is {start.isot}")
            # print(f"Exposure time for {l} is {image_info.requestedExposureTime[l]}")
            # print(f"finish is for {l} is {finish.isot}")
            start_vline = Span(location=start.datetime64, dimension='height', line_color='purple', line_width=0.5, line_dash='dashed')
            finish_vline = Span(location=finish.datetime64, dimension='height', line_color='blue', line_width=0.5, line_dash='dashed')
            
            ylabel_pos = np.median(yaxis_data)
            seq_num_label = Label(x=start.datetime64, y=ylabel_pos, text=str(image_info.imageNumber[l]))

            p.add_layout(start_vline)
            p.add_layout(finish_vline)
            p.add_layout(seq_num_label)

### Plot commanded position (by pointing component), target position by ATMCS, Calculated position by ATMCS

In [48]:
#derive principal (left) plot axis range
yr_cen=np.median(commanded_az_ATPng['azimuth'])
dy=1.1*(np.max(commanded_az_ATPng['azimuth'])- np.min(commanded_az_ATPng['azimuth']))

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth (degrees)"
p.xaxis.axis_label = "Time"

# Measured AZ from encoder0
p.cross(x=(Time(az.index.values)).value, 
        y=az['azimuthEncoderPosition0'], 
        color='red', line_width=1, line_dash='dashed', 
        legend_label='MTMount Measured Az Position')
# MTMount Commanded AZ from MTPtng
p.line(x=(Time(commanded_az_ATPng.index.values)).value, 
       y=commanded_az_ATPng['azimuth'], 
       color='green', line_width=1, line_dash='solid',
       legend_label='MTPtng Commanded Target Az')

# plot image boundaries
show_image_boundaries(image_info, commanded_az_ATPng['azimuth'])

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Azimuth velocity analysis

In [50]:
# Measured Velocity
measured_vel_az = await client.select_time_series("lsst.sal.MTMount.azimuth", 
                                                  ["actualVelocity", ], 
                                                  t1, t2)

# Commanded velocity from MTPointing Component
commanded_vel_az_ATPng = await client.select_time_series("lsst.sal.MTMount.command_trackTarget", 
                                                         ["azimuthVelocity","private_sndStamp", "private_rcvStamp"], 
                                                         t1, t2)

In [51]:
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth (degrees)"
p.xaxis.axis_label = "Time"

# AZ Positions
# Measured AZ from encoder0
p.cross(x=(Time(az.index.values)).value, 
        y=az['azimuthEncoderPosition0'], 
        color='red', line_width=1, line_dash='dashed', 
        legend_label='MTMount Measured Az Position')
# MTMount Commanded AZ from MTPtng
p.line(x=(Time(commanded_az_ATPng.index.values)).value, 
       y=commanded_az_ATPng['azimuth'], 
       color='green', line_width=1, line_dash='solid',
       legend_label='MTPtng Commanded Target Az')

# AZ Velocities
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
# Measured AZ velocity
p.cross(x=(Time(measured_vel_az.index.values)).value, 
        y=measured_vel_az['actualVelocity'], 
        color='blue', alpha=0.5, y_range_name='Velocity', 
        legend_label='MTMount Measured Az Velocity')
# vs commanded AZ velocity
p.line(x=(Time(commanded_vel_az_ATPng.index.values)).value, 
       y=commanded_vel_az_ATPng['azimuthVelocity'], 
       color='black', alpha=0.7, y_range_name='Velocity', 
       legend_label='MTPtng Commanded Target Az Velocity')


p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

### Azimuth Torque Analysis

In [ ]:
# Demanded and Measured Torques
# Motor 1
az_motor1_torque_demand = await client.select_packed_time_series("lsst.sal.ATMCS.torqueDemand", ["azimuthMotor1Torque", ], t1, t2)
az_motor1_torque_measured = await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", ["azimuthMotor1Torque", ], t1, t2)
# Motor 2
az_motor2_torque_demand = await client.select_packed_time_series("lsst.sal.ATMCS.torqueDemand", ["azimuthMotor2Torque", ], t1, t2)
az_motor2_torque_measured = await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", ["azimuthMotor2Torque", ], t1, t2)

In [ ]:
# Motor 1 Plot
yr_cen=np.median(measured_vel_az1['azimuthMotor1Velocity'])
dy=1.1*(np.max(measured_vel_az1['azimuthMotor1Velocity'])- np.min(measured_vel_az1['azimuthMotor1Velocity']))

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth Velocity (deg/s)"
p.xaxis.axis_label = "Time"
# Velocities
p.line(x=(Time(commanded_vel_az.index.values)).value, y=commanded_vel_az['azimuthVelocity'], color='black', alpha=0.7, legend_label='ATMCS Commanded Az Velocity Trajectory')
p.line(x=(Time(measured_vel_az1.index.values)).value, y=measured_vel_az1['azimuthMotor1Velocity'], color='red', alpha=0.5, legend_label='ATMCS Measured Az Velocity Motor 1')
times=Time((commanded_az_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_az_ATPng.index.values)).value
p.x(x=times, y=commanded_vel_az_ATPng['azimuthVelocity'], color='green', line_width=2, legend_label='ATPng Target Az Velocity')
# Torques
yr_cen=np.median(az_motor1_torque_demand['azimuthMotor1Torque'])
dy=(np.max(az_motor1_torque_demand['azimuthMotor1Torque']) - np.min(az_motor1_torque_demand['azimuthMotor1Torque']))/2
p.extra_y_ranges = {'Torque': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque [A]'), 'right')
# motor 1
p.line(x=(Time(az_motor1_torque_demand.index.values)).value, y=az_motor1_torque_demand['azimuthMotor1Torque'], color='orange', alpha=0.7, y_range_name='Torque', legend_label='ATMCS Commanded Motor 1 Torque')
p.line(x=(Time(az_motor1_torque_measured.index.values)).value, y=az_motor1_torque_measured['azimuthMotor1Torque'], color='blue', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Motor 1 Torque')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
# Motor 2 Plot
yr_cen=np.median(measured_vel_az2['azimuthMotor2Velocity'])
dy=1.1*(np.max(measured_vel_az2['azimuthMotor2Velocity'])- np.min(measured_vel_az2['azimuthMotor2Velocity']))

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth Velocity (deg/s)"
p.xaxis.axis_label = "Time"
# Velocities
p.line(x=(Time(commanded_vel_az.index.values)).value, y=commanded_vel_az['azimuthVelocity'], color='black', alpha=0.7, legend_label='ATMCS Commanded Az Velocity Trajectory')
p.line(x=(Time(measured_vel_az2.index.values)).value, y=measured_vel_az2['azimuthMotor2Velocity'], color='blue', alpha=0.5, legend_label='ATMCS Measured Az Velocity Motor 2')
times=Time((commanded_az_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_az_ATPng.index.values)).value
p.x(x=times, y=commanded_vel_az_ATPng['azimuthVelocity'], color='green', line_width=2, legend_label='ATPng Target Az Velocity')
# Torques
yr_cen=np.median(az_motor2_torque_demand['azimuthMotor2Torque'])
dy=(np.max(az_motor2_torque_demand['azimuthMotor2Torque']) - np.min(az_motor2_torque_demand['azimuthMotor2Torque']))/2
p.extra_y_ranges = {'Torque': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque [A]'), 'right')
# Motor 2
p.line(x=(Time(az_motor2_torque_demand.index.values)).value, y=az_motor2_torque_demand['azimuthMotor2Torque'], color='orange', alpha=0.7, y_range_name='Torque', legend_label='ATMCS Commanded Motor 2 Torque')
p.line(x=(Time(az_motor2_torque_measured.index.values)).value, y=az_motor2_torque_measured['azimuthMotor2Torque'], color='blue', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Motor 2 Torque')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

# Elevation Axis Analysis

In [ ]:
# Measured Position
el = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", ["elevationCalculatedAngle", ], t1, t2)

In [ ]:
# Commanded Trajectory
commanded_el = await client.select_packed_time_series("lsst.sal.ATMCS.trajectory", ["elevation", ], t1, t2)

In [ ]:
# Pointing Kernel Command
commanded_el_ATPng = await client.select_time_series("lsst.sal.ATMCS.command_trackTarget", ["elevation", "private_sndStamp", "private_rcvStamp",], t1, t2)

In [ ]:
# Plot positions
yr_cen=np.median(commanded_el['elevation'])
dy=1.1*(np.max(commanded_el['elevation'])-np.min(commanded_el['elevation']))/2

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Elevation Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=(Time(commanded_el.index.values)).value, y=commanded_el['elevation'], color='black', line_width=2, legend_label='ATMCS Commanded Elevation Trajectory')
p.cross(x=(Time(el.index.values)).value, y=el['elevationCalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Elevation Measured Elevation Position')
times=Time((commanded_el_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_el_ATPng.index.values)).value
p.cross(x=times, y=commanded_el_ATPng['elevation'], color='green', line_width=2, legend_label='ATPng Target Elevation')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Elevation Motor velocity Analysis
Note that while tracking, the velocity demand has an offset with respect to the actual velocity, this is normal as the velocity loop has no integrator (just a fixed gain). 

In [ ]:
#Measured
measured_vel_el = await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", ["elevationMotorVelocity", ], t1, t2)

In [ ]:
# Commanded
commanded_vel_el = await client.select_packed_time_series("lsst.sal.ATMCS.trajectory", ["elevationVelocity", ], t1, t2)

In [ ]:
# From Pointing
commanded_vel_el_ATPng = await client.select_time_series("lsst.sal.ATMCS.command_trackTarget", ["elevationVelocity", "private_sndStamp", "private_rcvStamp"], t1, t2)

In [ ]:
# Plot Velocities and Positions
yr_cen=np.median(commanded_el['elevation'])
dy=1.1*(np.max(commanded_el['elevation'])-np.min(commanded_el['elevation']))/2
# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Elevation Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=(Time(commanded_el.index.values)).value, y=commanded_el['elevation'], color='black', line_width=2, legend_label='ATMCS Commanded Elevation Trajectory')
p.cross(x=(Time(el.index.values)).value, y=el['elevationCalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Elevation Measured Elevation Position')
times=Time((commanded_el_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_el_ATPng.index.values)).value
p.cross(x=times, y=commanded_el_ATPng['elevation'], color='green', line_width=2, legend_label='ATPng Target Elevation')
# Velocities
yr_cen=np.median(measured_vel_el['elevationMotorVelocity'])
dy=1.1*(np.max(measured_vel_el['elevationMotorVelocity'])- np.min(measured_vel_el['elevationMotorVelocity']))/2

p.extra_y_ranges = {'Velocity': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=(Time(commanded_vel_el.index.values)).value, y=commanded_vel_el['elevationVelocity'], color='black', alpha=0.7, y_range_name='Velocity', legend_label='ATMCS Commanded Elevation Velocity Trajectory')
p.line(x=(Time(measured_vel_el.index.values)).value, y=measured_vel_el['elevationMotorVelocity'], color='red', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Measured Elevation Velocity')
p.x(x=times, y=commanded_vel_el_ATPng['elevationVelocity'], y_range_name='Velocity', color='green', line_width=2, legend_label='ATPng Target Elevation Velocity')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
# Query Torques
el_torque_demand = await client.select_packed_time_series("lsst.sal.ATMCS.torqueDemand", ["elevationMotorTorque", ], t1, t2)
el_torque_measured = await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", ["elevationMotorTorque", ], t1, t2)

In [ ]:
# Plot Positions versus torques
yr_cen=np.median(commanded_el['elevation'])
dy=1.1*(np.max(commanded_el['elevation'])-np.min(commanded_el['elevation']))/2
# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Elevation Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=(Time(commanded_el.index.values)).value, y=commanded_el['elevation'], color='black', line_width=2, legend_label='ATMCS Commanded Elevation Trajectory')
p.cross(x=(Time(el.index.values)).value, y=el['elevationCalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Elevation Measured Elevation Position')
times=Time((commanded_el_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_el_ATPng.index.values)).value
p.cross(x=times, y=commanded_el_ATPng['elevation'], color='green', line_width=2, legend_label='ATPng Target Elevation')

# Torques
yr_cen=np.median(el_torque_demand['elevationMotorTorque'])
dy=1.1*(np.max(el_torque_demand['elevationMotorTorque']) - np.min(el_torque_demand['elevationMotorTorque']))/2
p.extra_y_ranges = {'Torque': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque [A]'), 'right')
p.line(x=(Time(el_torque_demand.index.values)).value, y=el_torque_demand['elevationMotorTorque'], color='black', alpha=0.7, y_range_name='Torque', legend_label='ATMCS Commanded Torque')
p.line(x=(Time(el_torque_measured.index.values)).value, y=el_torque_measured['elevationMotorTorque'], color='red', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Torque')


p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
# plot
yr_cen=np.median(commanded_vel_el['elevationVelocity'])
dy=1.1*(np.max(commanded_vel_el['elevationVelocity']) - np.min(commanded_vel_el['elevationVelocity']))/2

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Elevation Velocity (deg/s)"
p.xaxis.axis_label = "Time"
# Velocities
p.line(x=(Time(commanded_vel_el.index.values)).value, y=commanded_vel_el['elevationVelocity'], color='black', alpha=0.7, legend_label='ATMCS Commanded Elevation Velocity Trajectory')
p.line(x=(Time(measured_vel_el.index.values)).value, y=measured_vel_el['elevationMotorVelocity'], color='red', alpha=0.5, legend_label='ATMCS Measured Elevation Velocity')
times=Time((commanded_el_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_el_ATPng.index.values)).value
p.x(x=times, y=commanded_vel_el_ATPng['elevationVelocity'], color='green', 
    line_width=2, legend_label='ATPng Target Elevation Velocity')

# Torques
yr_cen=np.median(el_torque_demand['elevationMotorTorque'])
dy=1.1*(np.max(el_torque_demand['elevationMotorTorque']) - np.min(el_torque_demand['elevationMotorTorque']))/2
p.extra_y_ranges = {'Torque': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque [A]'), 'right')
p.line(x=(Time(el_torque_demand.index.values)).value, y=el_torque_demand['elevationMotorTorque'], color='pink', alpha=0.7, y_range_name='Torque', legend_label='ATMCS Commanded Torque')
p.line(x=(Time(el_torque_measured.index.values)).value, y=el_torque_measured['elevationMotorTorque'], color='blue', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Torque')


p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

# Nasmyth 1 Analysis
#### Note that if Nasmyth 1 is rarely used and depending on the timestamps chosen this may fail. 
#### Nasmyth 2 is where LATISS is installed and the same plots are available below

In [ ]:
# Measured Position
nas1 = await client.select_packed_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", ["nasmyth1CalculatedAngle", ], t1, t2)

In [ ]:
# Commanded Trajectory
commanded_nas1 = await client.select_packed_time_series("lsst.sal.ATMCS.trajectory", ["nasmyth1RotatorAngle", ], t1, t2)

In [ ]:
# Pointing Kernel Command
commanded_nas1_ATPng = await client.select_time_series("lsst.sal.ATMCS.command_trackTarget", ["nasmyth1RotatorAngle", "private_sndStamp", "private_rcvStamp", ], t1, t2)

## Plot commanded position (by pointing component), target position by ATMCS, Calculated position by ATMCS

In [ ]:
yr_cen=np.median(commanded_nas1['nasmyth1RotatorAngle'])
dy=1.1*(np.max(commanded_nas1['nasmyth1RotatorAngle']) - np.min(commanded_nas1['nasmyth1RotatorAngle']))/2

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=(Time(commanded_nas1.index.values)).value, y=commanded_nas1['nasmyth1RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas1 Trajectory')
p.cross(x=(Time(nas1.index.values)).value, y=nas1['nasmyth1CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Measured Nas1 Position')
times=Time((commanded_nas1_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_nas1_ATPng.index.values)).value
p.cross(x=times, y=commanded_nas1_ATPng['nasmyth1RotatorAngle'], color='green', line_width=5, legend_label='ATPng Target Nas1')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Look at Nasmyth 1 motor velocities
#### Note that while tracking, the velocity demand has an offset with respect to the actual velocity, this is normal as the velocity loop has no integrator (just a fixed gain). 

In [ ]:
#Measured
measured_vel_nas1 = await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", ["nasmyth1MotorVelocity", ], t1, t2)
# Commanded
commanded_vel_nas1 = await client.select_packed_time_series("lsst.sal.ATMCS.trajectory", ["nasmyth1RotatorAngleVelocity", ], t1, t2)
# From Pointing
commanded_vel_nas1_ATPng = await client.select_time_series("lsst.sal.ATMCS.command_trackTarget", ["nasmyth1RotatorAngleVelocity", "private_sndStamp", "private_rcvStamp", ], t1, t2)

In [ ]:
yr_cen=np.median(commanded_nas1['nasmyth1RotatorAngle'])
dy=1.1*(np.max(commanded_nas1['nasmyth1RotatorAngle']) - np.min(commanded_nas1['nasmyth1RotatorAngle']))/2

# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=(Time(commanded_nas1.index.values)).value, y=commanded_nas1['nasmyth1RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas1 Trajectory')
p.cross(x=(Time(nas1.index.values)).value, y=nas1['nasmyth1CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Nas1 Measured Nas1 Position')
times=Time((commanded_nas1_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_nas1_ATPng.index.values)).value
p.cross(x=times, y=commanded_nas1_ATPng['nasmyth1RotatorAngle'], color='green', line_width=2, legend_label='ATPng Target Nas1')

# Velocities
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=(Time(commanded_vel_nas1.index.values)).value, y=commanded_vel_nas1['nasmyth1RotatorAngleVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Commanded Nas1 Velocity Trajectory')
p.line(x=(Time(measured_vel_nas1.index.values)).value, y=measured_vel_nas1['nasmyth1MotorVelocity'], color='blue', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Measured Nas1')
p.cross(x=times, y=commanded_vel_nas1_ATPng['nasmyth1RotatorAngleVelocity'], y_range_name='Velocity', color='green', line_width=2, legend_label='ATPng Target Nas1 Velocity')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

## Look at Nasmyth 1 Torques as a function of position and velocity

In [ ]:
# Query Torques
nas1_torque_demand = await client.select_packed_time_series("lsst.sal.ATMCS.torqueDemand", ["nasmyth1MotorTorque", ], t1, t2)
nas1_torque_measured = await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", ["nasmyth1MotorTorque", ], t1, t2)

In [ ]:
# Plot Position and torque as a function of time
yr_cen=np.median(commanded_nas1['nasmyth1RotatorAngle'])
dy=1.1*(np.max(commanded_nas1['nasmyth1RotatorAngle']) - np.min(commanded_nas1['nasmyth1RotatorAngle']))/2

# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"

p.line(x=commanded_nas1.index.values, y=commanded_nas1['nasmyth1RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas1 Trajectory')
p.cross(x=nas1.index.values, y=nas1['nasmyth1CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Nas1 Measured Nas1 Position')
times=Time((commanded_nas1_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_nas1_ATPng.index.values)).value
p.cross(x=times, y=commanded_nas1_ATPng['nasmyth1RotatorAngle'], color='green', line_width=2, legend_label='ATPng Target Nas1')

# Torques
yr_cen=np.median(nas1_torque_measured['nasmyth1MotorTorque'])
dy=1.1*(np.max(nas1_torque_measured['nasmyth1MotorTorque']) - np.min(nas1_torque_measured['nasmyth1MotorTorque']))/2

p.extra_y_ranges = {'Torque': Range1d(yr_cen-dy, yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque'), 'right')
p.line(x=nas1_torque_demand.index.values, y=nas1_torque_demand['nasmyth1MotorTorque'], color='black', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Demanded Nas1 Torque')
p.line(x=nas1_torque_measured.index.values, y=nas1_torque_measured['nasmyth1MotorTorque'], color='blue', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Nas1 Torque')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
smooth_window=10 # set to 1 for no smoothing

# Plot velocity and torque as a function of time
yr_cen=np.median(commanded_vel_nas1['nasmyth1RotatorAngleVelocity'])
dy=10.1*(np.max(commanded_vel_nas1['nasmyth1RotatorAngleVelocity']) - np.min(commanded_vel_nas1['nasmyth1RotatorAngleVelocity']))/2

# Velocities
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Velocity (deg/s)"
p.xaxis.axis_label = "Time"

x1 = commanded_vel_nas1.index.values ; y1 = commanded_vel_nas1['nasmyth1RotatorAngleVelocity'].rolling(window=smooth_window).mean()
x2 = measured_vel_nas1.index.values ; y2 = measured_vel_nas1['nasmyth1MotorVelocity'].rolling(window=smooth_window).mean()
x3 = nas1_torque_demand.index.values ; y3 = nas1_torque_demand['nasmyth1MotorTorque'].rolling(window=smooth_window).mean()
x4 = nas1_torque_measured.index.values ; y4 = nas1_torque_measured['nasmyth1MotorTorque'].rolling(window=smooth_window).mean()

p.line(x=x1, y=y1, color='black', alpha=0.5, legend_label='ATMCS Commanded Nas1 Velocity Trajectory')
p.line(x=x2, y=y2, color='red', alpha=0.5, legend_label='ATMCS Measured Nas1 Velocity Motor')
times=Time((commanded_vel_nas1_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_vel_nas1_ATPng.index.values)).value
p.x(x=times, y=commanded_vel_nas1_ATPng['nasmyth1RotatorAngleVelocity'], color='green', line_width=2, legend_label='ATPng Target Nas1 Velocity')


# Torques
yr_cen=np.median(nas1_torque_measured['nasmyth1MotorTorque'])
dy=1.1*(np.max(nas1_torque_measured['nasmyth1MotorTorque']) - np.min(nas1_torque_measured['nasmyth1MotorTorque']))/2

p.extra_y_ranges = {'Torque': Range1d(yr_cen-dy, yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque'), 'right')
p.line(x=x3, y=y3, color='black', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Demanded Nas1 Torque')
p.line(x=x4, y=y4, color='blue', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Nas1 Torque')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

# Nasmyth 2 Analysis
#### Note that if Nasmyth 1 was not in use during the selected timestamp this will fail. Nasmyth 1 plots are available above

In [ ]:
# Measured Position
nas2 = await client.select_packed_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", ["nasmyth2CalculatedAngle", ], t1, t2)

In [ ]:
# Commanded Trajectory
commanded_nas2 = await client.select_packed_time_series("lsst.sal.ATMCS.trajectory", ["nasmyth2RotatorAngle", ], t1, t2)

In [ ]:
# Pointing Kernel Command
commanded_nas2_ATPng = await client.select_time_series("lsst.sal.ATMCS.command_trackTarget", ["nasmyth2RotatorAngle", "private_sndStamp", "private_rcvStamp", ], t1, t2)

## Plot commanded position (by pointing component), target position by ATMCS, Calculated position by ATMCS

In [ ]:
yr_cen=np.median(commanded_nas2['nasmyth2RotatorAngle'])
dy=1.1*(np.max(commanded_nas2['nasmyth2RotatorAngle']) - np.min(commanded_nas2['nasmyth2RotatorAngle']))/2

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=(Time(commanded_nas2.index.values)).value, y=commanded_nas2['nasmyth2RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas2 Trajectory')
p.cross(x=(Time(nas2.index.values)).value, y=nas2['nasmyth2CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Measured Nas2 Position')#
times=Time((commanded_nas2_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_nas2_ATPng.index.values)).value
p.cross(x=times, y=commanded_nas2_ATPng['nasmyth2RotatorAngle'], color='green', line_width=5, legend_label='ATPng Target Nas2')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Look at Nasmyth 2 motor velocities
#### Note that while tracking, the velocity demand has an offset with respect to the actual velocity, this is normal as the velocity loop has no integrator (just a fixed gain). 

In [ ]:
#Measured
measured_vel_nas2 = await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", ["nasmyth2MotorVelocity", ], t1, t2)
# Commanded
commanded_vel_nas2 = await client.select_packed_time_series("lsst.sal.ATMCS.trajectory", ["nasmyth2RotatorAngleVelocity", ], t1, t2)
# From Pointing
commanded_vel_nas2_ATPng = await client.select_time_series("lsst.sal.ATMCS.command_trackTarget", ["nasmyth2RotatorAngleVelocity", "private_sndStamp", "private_rcvStamp", ], t1, t2)

In [ ]:
# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
yr_cen=np.median(commanded_nas2['nasmyth2RotatorAngle'])
dy=1.1*(np.max(commanded_nas2['nasmyth2RotatorAngle']) - np.min(commanded_nas2['nasmyth2RotatorAngle']))/2
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"

p.line(x=(Time(commanded_nas2.index.values)).value, y=commanded_nas2['nasmyth2RotatorAngle'], color='black', line_width=2, legend_label='ATMCS Commanded Nas2 Trajectory')
p.cross(x=(Time(nas2.index.values)).value, y=nas2['nasmyth2CalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Nas1 Measured Nas2 Position')
times=Time((commanded_vel_nas2_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_vel_nas2_ATPng.index.values)).value
p.cross(x=times, y=commanded_nas2_ATPng['nasmyth2RotatorAngle'], color='green', line_width=2, legend_label='ATPng Target Nas2')

# Velocities
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=(Time(commanded_vel_nas2.index.values)).value, y=commanded_vel_nas2['nasmyth2RotatorAngleVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Commanded Nas2 Velocity Trajectory')
p.line(x=(Time(measured_vel_nas2.index.values)).value, y=measured_vel_nas2['nasmyth2MotorVelocity'], color='blue', alpha=0.5, y_range_name='Velocity', legend_label='ATMCS Measured Nas2 Velocity')
p.x(x=times, y=commanded_vel_nas2_ATPng['nasmyth2RotatorAngleVelocity'], y_range_name='Velocity', color='orange', line_width=2, legend_label='ATPng Target Nas2 Velocity')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

### Nasmyth 2 Torque Analysis

In [ ]:
# Query Torques
nas2_torque_demand = await client.select_packed_time_series("lsst.sal.ATMCS.torqueDemand", ["nasmyth2MotorTorque", ], t1, t2)
nas2_torque_measured = await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", ["nasmyth2MotorTorque", ], t1, t2)

In [ ]:
smooth_window=10 # set to 1 for no smoothing

# Plot Position and torque as a function of time
yr_cen=np.median(commanded_nas2['nasmyth2RotatorAngle'])
dy=1.1*(np.max(commanded_nas2['nasmyth2RotatorAngle']) - np.min(commanded_nas2['nasmyth2RotatorAngle']))/2

# Positions
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Position (degrees)"
p.xaxis.axis_label = "Time"

x1=commanded_nas2.index.values ; y1=commanded_nas2['nasmyth2RotatorAngle'].rolling(window=smooth_window).mean()
x2=nas2.index.values; y2 = nas2['nasmyth2CalculatedAngle'].rolling(window=smooth_window).mean()
x3=nas2_torque_demand.index.values ; y3=nas2_torque_demand['nasmyth2MotorTorque'].rolling(window=smooth_window).mean()
x4=nas2_torque_measured.index.values ; y4=nas2_torque_measured['nasmyth2MotorTorque'].rolling(window=smooth_window).mean()

p.line(x=x1, y=y1, color='black', line_width=2, legend_label='ATMCS Commanded Nas2 Trajectory')
p.cross(x=x2, y=y2, color='red', line_width=2, line_dash='dashed', legend_label='ATMCS Measured Nas2 Position')
times=Time((commanded_vel_nas2_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_vel_nas2_ATPng.index.values)).value
p.cross(x=times, y=commanded_nas2_ATPng['nasmyth2RotatorAngle'], color='green', line_width=2, legend_label='ATPng Target Nas2')

# Torques
yr_cen=np.median(nas2_torque_measured['nasmyth2MotorTorque'])
dy=1.1*(np.max(nas2_torque_measured['nasmyth2MotorTorque']) - np.min(nas2_torque_measured['nasmyth2MotorTorque']))/2

p.extra_y_ranges = {'Torque': Range1d(yr_cen-dy, yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque'), 'right')

p.line(x=x3, y=y3, color='orange', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Demanded Nas2 Torque')

p.line(x=x4, y=y4, color='blue', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Nas2 Torque')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

### Nasmyth 2 Torque and velocity Analysis

In [ ]:
smooth_window=10 # set to 1 for no smoothing

# Plot velocity and torque as a function of time
yr_cen=np.median(commanded_vel_nas2['nasmyth2RotatorAngleVelocity'])
dy=10.1*(np.max(commanded_vel_nas2['nasmyth2RotatorAngleVelocity']) - np.min(commanded_vel_nas2['nasmyth2RotatorAngleVelocity']))/2

# Velocities
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth Velocity (deg/s)"
p.xaxis.axis_label = "Time"

x1 = commanded_vel_nas2.index.values ; y1 = commanded_vel_nas2['nasmyth2RotatorAngleVelocity'].rolling(window=smooth_window).mean()
x2 = measured_vel_nas2.index.values ; y2 = measured_vel_nas2['nasmyth2MotorVelocity'].rolling(window=smooth_window).mean()
x3 = nas2_torque_demand.index.values ; y3 = nas2_torque_demand['nasmyth2MotorTorque'].rolling(window=smooth_window).mean()
x4 = nas2_torque_measured.index.values ; y4 = nas2_torque_measured['nasmyth2MotorTorque'].rolling(window=smooth_window).mean()

p.line(x=x1, y=y1, color='black', alpha=0.5, legend_label='ATMCS Commanded Nas2 Velocity Trajectory')
p.line(x=x2, y=y2, color='red', alpha=0.5, legend_label='ATMCS Measured Nas2 Velocity Motor')
times=Time((commanded_vel_nas2_ATPng['private_sndStamp']), format='unix')
times=(Time(commanded_vel_nas2_ATPng.index.values)).value
p.x(x=times, y=commanded_vel_nas2_ATPng['nasmyth2RotatorAngleVelocity'], color='green', line_width=2, legend_label='ATPng Target Nas2 Velocity')


# Torques
yr_cen=np.median(nas2_torque_measured['nasmyth2MotorTorque'])
dy=1.1*(np.max(nas2_torque_measured['nasmyth2MotorTorque']) - np.min(nas2_torque_measured['nasmyth2MotorTorque']))/2

p.extra_y_ranges = {'Torque': Range1d(yr_cen-dy, yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque'), 'right')
p.line(x=x3, y=y3, color='black', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Demanded Nas2 Torque')
p.line(x=x4, y=y4, color='blue', alpha=0.5, y_range_name='Torque', legend_label='ATMCS Measured Nas2 Torque')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)